# 19 Animation

우리가 사용하는 컴퓨팅 환경은 결코 정적이지 안습니다. 현대의 컴퓨터는 다양한 화면효과를 여러가지 형태로 표현하고 있으며 시간에 따라 화면의 여로 요소들이 지속적인 변화를 표현합니다. 물론 사용자의 행동(마우스 클릭등)에 의한 대응으로 변화를 표현 할 수도 있습니다만 경우에 따라서는 사용자의 개입 없이도 주기적으로 변화를 가함으로서 시각적인 효과를 줍니다. 이런 효과들을 애니메이션이라고 합니다

이러한 애니메이션의 원리는 컴퓨터의 내부 시계가 대단히 미세한 간격으로 인터럽트를 발생시키도록 함으로서 이뤄집니다. 원래 컴퓨터라는 연속계산기는 프로그램의 순서와 분기에 의해서 정해진 작동을 수행합니다. 하지만 이런 경우 외부 장치에서 오는 신호를 주기적으로 체크해야 하는데 이를 너무 자주 하면 프로그램의 실행에 부담이 너무 크며 , 너무 가끔하면 대응이 너무 느려지게 됩니다. 그래서 하드웨어적으로 외부장치에서 신호가 오면 자동으로 작동을 멈추고 등록된 처리루틴으로 분기를 시키는데 이를 인터럽트라고 합니다. 

ipywidgets 이나 tkinter 에서 발생하는 마우스 이벤트도 매번 마우스의 상태를 프로그램이 직접 검사하고 처리하는게 아니라 마우스를 이동하거나 클릭하면 관련 프로그램이 자동으로 실행하도록 미리 등록해두는 식으로 처리합니다. 그리고 외부 장치의 변화 없이도 일정 시간주기로 이러한 인터럽트를 발생시킬 수 있는데 이를 타이머 인터럽트라고 합니다. 


### 19.1 tkinter 의 timer 설정 

tkinter 는 스스로 윈도우 창을 만들고 이에 관련된 이벤트들을 처리하기 때문에 타이머 이벤트도 직접 관리합니다. 그러므로 다음과 같이 일정 시간뒤에 작동하는 프로그램을 만들 수 있습니다. 


In [1]:
import tkinter as tk 
root = tk.Tk()

def delayprint():
    print('!')

root.after(1000, delayprint) # 1초 뒤에 delayprint를 실행하도록 등록
tk.mainloop()


!


위의 예에서 delayprint() 는 1초뒤에 한번 작동하고 그 후에는 작동하지 안습니다.  만일 위의 코드가 1초마다 계속 작동하게 만들고싶다면 다음과 같이 수정하면 됩니다. 



In [2]:
import tkinter as tk 
root = tk.Tk()

def delayprint():
    print('!')
    root.after(1000,delayprint)
    
root.after(1000, delayprint) # 1초 뒤에 delayprint를 실행하도록 등록
tk.mainloop()

!
!
!


이제 위의 예제를 변경해서 1초마다 카운트다운을 하는 창을 만들어보도록 하겠습니다. 

In [3]:
root = tk.Tk() 
cnt = 10
label = tk.Label(root, text = str(cnt), width = 6, height = 3, font = ('Arial',40))
label.pack() 

def count():
    global cnt 
    cnt -= 1        
    label['text'] = str(cnt)
    root.after(1000, count)
    
root.after(1000,count) 
tk.mainloop() 


### 19.2 Canvas 를 이용한 애니메이션 

이번에는 위의 after를 사용하여  와 Canvas 위에서 원이 움직이는 애니메이션을 한번 만들어보도록 하겠습니다. 

canvas 에서 원을 생성하고 이동시킬 경우 canvas 위젯의 move 함수를 사용합니다. 

```python
ball = cvs.create_oval(100,100,200,200)
cvs.move(ball, 3,3) # 원을 x,y 각각 3,3 만큼 이동시킴
```

이를 이용해서 공이 대각선으로 움직이는 코드는 다음과 같습니다. 

In [5]:
root = tk.Tk() 
cvs = tk.Canvas(width = 600, height = 600, bg = 'white')
cvs.pack()
ball = cvs.create_oval(100,100,200,200, fill = 'blue', outline = '')
def move():
    cvs.move(ball, 3,3)
    root.after(10, move)
move() 
tk.mainloop()

물론 move 만 사용해야 하는 것은 아닙니다.  선이 위치를 옮기도록 설정할 수도 있습니다. 

In [7]:
root = tk.Tk() 
cvs = tk.Canvas(width = 600, height = 600, bg = 'white')
cvs.pack()
line = cvs.create_line(100,100,200,200)
def move():
    cds = cvs.coords(line)
    cvs.coords(line, cds[0],cds[1],cds[2], cds[3] + 1 )    
    root.after(10, move)
move() 
tk.mainloop()

### 19.3 바운싱 볼 


#### 19.3.1 기본적인 바운싱 볼

간단한 애니메이션의 유명한 예제로 bouncing ball 이라는 예제가 있습니다. 공이 아래로 떨어지다가 다시 튕겨오르는 예제죠. 

이는 물리학에서 속도와 중력가속도를 이용해 간단히 구현 가능합니다. 

v : 속도 
g : 중력가속도

이때 매 순간마다 속도에 중력가속도를 더 해 속도를 변경시키고 이를 이용해 공의 움직임을 묘사할 수 있습니다. 

```python
v += g  
cvs.move(ball, 0, v ) 
```

이제 공이 튕기는걸 표현해야 하는데 공이 바닦에 닿은 순간을 알아내야 합니다. 

cvs.cooords(ball) ->  [원의 좌측 x, 원의 위쪽 y, 원의 우측 x, 원의 아랫쪽y ]

공이 cvs.coords(ball)[3] 이 원의 바닦의 y값이 되며 이것이 canvas 의 최하단 좌표인 600을 넘을 경우 v 값을 음수로 전환함으로서 튕김을 표현할 수 있습니다. 

```python 
if cvs.coords[3] > 600: 
    v *= -1
```

이제 이를 이용해 바닦에 튀는 공을 애니메이션으로 만들어봅니다. 


In [14]:
from random import random 

root = tk.Tk()
cvs = tk.Canvas(root, width = 600, height = 600, bg = '#FFFFFF')
cvs.pack()

ball = cvs.create_oval(100,100,200,200, fill = 'blue')
v, g = 0, 0.2 

def fall():
    global v,g    
    if cvs.coords(ball)[3] > 600:
        v *= -1
    else:
        v += g            
    cvs.move(ball,0,v)
    root.after(30,fall)    
fall()

tk.mainloop()


#### 19.3.2 3개의 바운싱 볼

이번에는 3개의 공이 튀는 예를 만들어보도록 하겠습니다. 단 각각 150 픽셀간격으로 생성하되 y는 100~300 에서 랜덤으로 생성하는 코드입니다. 이때 각각의 공의 속도도 따로 기록되어야 하기 때문에 v 도 배열로 만들어 초기값 0 을 넣어줍니다. 

```python 
balls,v = [],[]
for i in range(3):
    y = int(random() * 200) + 100 
    balls += [cvs.create_oval(100+i*150,y-50,200+i*150, y+50, fill = 'blue')]
    v += [0]
```

이제 배열화된 공을 각각 위의 코드처럼 각각의 속도변화를 처리하면 다음과 같이 됩니다. 

```python
for i in range(3):
    pos = cvs.coords(balls[i])
    if pos[3] > 600:
        v[i] *= -1 
    else:
        v[i] += g        
    cvs.move(balls[i],0,v[i])
```

이를 합쳐서 구현하면 다음과 같습니다. 

In [18]:
from random import random 

root = tk.Tk()
cvs = tk.Canvas(root, width = 600, height = 600, bg = '#FFFFFF')
cvs.pack()
balls,v = [],[]
for i in range(3):
    y = int(random() * 200) + 100 
    balls += [cvs.create_oval(100+i*150,y,200+i*150, y+100, fill = 'blue')]
    v += [0]
g = 0.2

def fall():
    global v,g,balls
    for i in range(3):
        pos = cvs.coords(balls[i])
        if pos[3] > 600:
            v[i] = v[i] * -1 
        else:
            v[i] += g        
        cvs.move(balls[i],0,v[i])
    root.after(30,fall)
fall()
tk.mainloop()



#### 19.3.3 100개의 바운싱 볼

이번에는 더 많은 공들(100개)을 생성하되 x축 방향의 움직임도 넣어주도록 하겠습니다. 대단히 복잡할 것같지만 x 의 좌표도 50~550사이에 랜덤생성하고 v 에 y 축 속도만 넣어주는 대신 x 축 속도를 -3 ~ +3 의 랜덤값으로 넣어줄 뿐 기본적으론 비슷합니다. 

단 단 공이 많기 때문에 rndcolor 함수를 이용해 각각 공의 색을 랜덤으로 생성해주도록 합니다. 

```python 
def rndcolor(): # '#a043e5 같은 16진수 색상값을 랜덤생성 ' 
    return '#%02x%02x%02x' % (int(random() * 255), int(random() * 255), int(random() * 255))

balls,v = [],[]
for i in range(100):
    x,y = int(random() * 500 + 50 ) , int(random() * 200) + 100 
    balls += [cvs.create_oval(x-25,y-25, x+25, y+25, fill = rndcolor() )] # 공의 크기는 50으로
    v += [[int(random()*6 -3), 0]]  # 각각 공의 속도는 [x속도, y속도] 로 나뉨
```

단 x 축도 벽에서 튕기기 위해서 coords[0] < 0 or coords[2] > 600 의 조건으로 x 속도도 변경시켜줍니다. 
```python 
coords = cvs.coords[ balls[i] ] # i번째 공의 위치값을 가져온다.
if coords[0] < 0 or coords[2] > 600 : # 공이 옆면에 부딪히면
    v[i][0] *= -1 # i번째 공의 x 소도를 반전시킨다. 
```

그러면 위의 코드를 적용해 x 축 움직임까지 포함하는 애니메이션으로 바꿔보겠습니다. 




In [28]:
from random import random 
def rndcolor():
    return '#%02x%02x%02x' % (int(random() * 255), int(random() * 255), int(random() * 255))

root = tk.Tk()
cvs = tk.Canvas(root, width = 600, height = 600, bg = '#FFFFFF')
cvs.pack()
g = 0.2
balls,v = [],[]

for i in range(100):
    x,y = int(random() * 500 + 50 ) , int(random() * 200) + 100 
    balls += [cvs.create_oval(x-25,y-25, x+25, y+25, fill = rndcolor() )]
    v += [[int(random()*6 -3),0]] 

def move():
    global v,g,balls
    for i in range(100):
        coords = cvs.coords( balls[i] ) # i번째 공의 위치값을 가져온다.
        if coords[0] < 0 or coords[2] > 600 : # 공이 옆면에 부딪히면
            v[i][0] *= -1
            
        if coords[3] > 600:
            v[i][1] *= -1
        else:
            v[i][1] += g
        
        cvs.move(balls[i],v[i][0],v[i][1])
    root.after(10,move)
move()
tk.mainloop()
     
    


#### 19.3.4 무중력 볼

아래는 중력가속도 없이  x 축의 움직임처럼 y축도 위아래로 튕기는 무중력 상태를 상정한 움직임으로 바꿔본 예제입니다. 


In [31]:
from random import random 
def rndcolor():
    return '#%02x%02x%02x' % (int(random() * 255), int(random() * 255), int(random() * 255))

root = tk.Tk()
cvs = tk.Canvas(root, width = 600, height = 600, bg = '#FFFFFF')
cvs.pack()

balls,v = [],[]

for i in range(100):
    x,y = int(random() * 500 + 50 ) , int(random() * 500) + 50  # x, y모두 화면 골고루 분포  
    balls += [cvs.create_oval(x-25,y-25, x+25, y+25, fill = rndcolor() )]
    v += [[int(random()*10 - 5),int(random()*10 -5)]] # x,y 전부 -5 에서 +5 사이의 랜덤속도

def move():
    global v,g,balls
    for i in range(100):
        coords = cvs.coords( balls[i] ) 
        if coords[0] < 0 or coords[2] > 600 : # 공이 옆면에 부딪히면
            v[i][0] *= -1            
        if coords[1] < 0 or coords[3] > 600 :  # 공이 위아래로 부딪히면
            v[i][1] *= -1
        
        cvs.move(balls[i],v[i][0],v[i][1])
    root.after(10,move)
move()
tk.mainloop()
     
    

### 19.4 눈내리기 표현

다음은 공과 비슷하지만 눈이 내리는 것을 표현하는 애니메이션입니다.  물론 단순하게 표현하면 랜덤한 공이 아래로 떨어지면 되지만 입체감을 주기 위해 거리값 d 를 생성하고 거리가 멀면 눈의 크기가 작아지며 속도도 더 느려진다는 것을 표현하여 거리에 대한 착시를 표현했습니다. 



In [33]:
from random import random
root = tk.Tk() 
cvs = tk.Canvas(root, width = 600, height = 600, bg = 'black')
cvs.pack()

snow,cnt  = [],100
info = {} 
for i in range(cnt):
    x,y,d = int( random() * 600 ), int( random() * 600 - 300 ), int(3 + random() * 7) # 거리d 는 3 ~ 10 
    r = int(30 / d)  # 거리에 따라 눈의 크기가 달라짐
    flake =  cvs.create_oval(x-r,y-r,x+r,y+r, fill = 'white', outline = '') 
    snow += [flake]
    info[flake]  = d  
    
def fall():
    for flake in snow:
        d = info[flake]        
        if cvs.coords(flake)[1] > 600:
            v = -800 # 눈이 아래로 화면을 넘어가면 다시 화면 상단으로 이동
        else:
            v = 10 / d  # 거리에 따라 10에서 속도가 10을 기준으로 점점 느려짐
        cvs.move(flake, 0, v)

        
    root.after(10,fall)

fall()
tk.mainloop()


### 19.5 시계 움직임 애니메이션

이번에는 시계 움직임을 한번 그려보도록 하겠습니다. 먼저 시계가 움직일 좌표를 위해 180도에서 -180도 사이를 60등분 하는 배열을 만듭니다. 프로그래밍에선 180도를 np.pi 포현하기 때문에 각도의 배열 theta는 다음과 같이 생성됩니다. 

```python 
import numpy as np 
theta = np.linspace(np.pi, -np.pi, 61) 
```

이때 각도별 길이가 1인 원의 x좌표는 중점을 중심으로 sin(theta) 값으로 , y좌표는 cos(theta)로 계산이 가능합니다. 
```python
x = np.sin(theta)
y = np.cos(theta)
```

이를 이용해 길이가 중심에서 원주를 도는 길이가 100인 선들을 그리면 다음과 같습니다. 

```python 
for i in range(60):
    cvs.create_line(200,200, 200+int(x[i]*100), 200+int(y[i]*100))
```

이를 타이머로 변경하고 0~59를 회전하는 t값에 대응하도록 선을 그리되 이전 선을 지우면
바늘이 회전하는 애니메이션이 됩니다. 


In [39]:
import numpy as np 
theta = np.linspace(np.pi, -np.pi, 61) 
x = np.sin(theta)
y = np.cos(theta)

root = tk.Tk()
cvs = tk.Canvas(root, width=400, height=400)
cvs.pack()
t = 0 
needle = cvs.create_line(200,200, 200+int(x[t]*100), 200+int(y[t]*100))
def draw():
    global t,needle 
    t = (t + 1)%60 # t 는 0~59 를 반복
    cvs.delete(needle) # 이전 라인을 지움
    needle = cvs.create_line(200,200, 200+int(x[t]*100), 200+int(y[t]*100))
    root.after(100,draw)
draw()
tk.mainloop()

이제 시계 눈금을 그려보도록 하겠습니다. 다음 코드처럼 선의 시작점을 중점이 아니라 길이가 100인 원주에서 110 원주까지 이으면 시계의 눈금이 생성됩니다. 
``` python
for i in range(60):
    cvs.create_line(
        200+int(x[i]*100), 200+int(y[i]*100),  
        200+int(x[i]*110), 200+int(y[i]*110)
    )
```

이때 5칸 단위로 굵고 좀더 긴 선을 그려주면 시간의 눈금도 표현이 가능합니다. 

``` python
for i in range(60):
    w,l = 1,110
    if i%5 == 0:
        w,l = 1,115
    cvs.create_line(
        200+int(x[i]*100), 200+int(y[i]*100),  
        200+int(x[i]*l), 200+int(y[i]*l),
        width = w        
    )
    
```

그러면 시계 눈금을 바늘의 움직임과 같이 해보겠습니다. 


In [47]:
import numpy as np 
theta = np.linspace(np.pi, -np.pi, 61) 
x = np.sin(theta)
y = np.cos(theta)

root = tk.Tk()
cvs = tk.Canvas(root, width=400, height=400)
cvs.pack()

for i in range(60):
    w,l = 1,110
    if i%5 == 0:
        w,l = 2,115
    cvs.create_line(
        200+int(x[i]*100), 200+int(y[i]*100),  
        200+int(x[i]*l), 200+int(y[i]*l),
        width = w        
    )
    
t = 0 

needle = cvs.create_line(200,200, 200+int(x[t]*100), 200+int(y[t]*100))
def draw():
    global t,needle 
    t = (t + 1)%60 # t 는 0~59 를 반복
    cvs.delete(needle) # 이전 라인을 지움
    needle = cvs.create_line(200,200, 200+int(x[t]*100), 200+int(y[t]*100))
    root.after(100,draw)
draw()
tk.mainloop()


이제 시침, 분침도 같이 추가해보겠습니다.  이를 위해 시간 t 대신 se,mi,hr 로 초,분,시 를 표현하도록 하고 60초마다 분이 올라가고 12분 마다 시간의 칸이 올라가도록 해줍니다. 

```python
sec = (sec + 1) % 60
if sec == 0 :
    mi = (mi + 1) % 60    
    if mi % 12 == 0 :
        hr = (hr + 1) % 60
```

코드를 단순화하기 위해서 바늘을 그리는 needle 함수를 만들고 시분초의 선을 nhr, nmi, nsec 로 구분합니다. 

```python 
nsec, nmi, nhr = needle(sec, 1, 100), needle(mi, 2, 95), needle(hr, 3, 75)
```

이제 시분초를 합쳐서 구현해보겠습니다. 시침의 움직임을 확인하기 위해 시간간격을 최대한 짧게 설정해줍니다. 


In [ ]:
# 시계 흉내내기 

import numpy as np 
theta = np.linspace(np.pi, -np.pi, 61)
x = np.sin(theta)
y = np.cos(theta)
root = tk.Tk()
cvs = tk.Canvas(root, width = 400, height = 400)
cvs.pack()
for i in range(60):
    w,l = 1,110
    if i % 5 == 0 :
        w,l = 3,120        
    cvs.create_line(200+int(x[i]*100), 200+int(y[i]*100),
                    200+int(x[i]*l), 200+int(y[i]*l),
                    width = w
                   )

def needle(t, w, l):
    return cvs.create_line(200,200,200+int(x[t]*l),200+int(y[t]*l), width = w)

sec, mi, hr = 0,0,0
nsec, nmi, nhr = needle(sec, 1, 100), needle(mi, 2, 95), needle(hr, 3, 75)

def draw():
    global sec,mi,hr,nsec,nmi,nhr
    sec = (sec + 1) % 60
    if sec == 0 :
        mi = (mi + 1) % 60
        cvs.delete(nmi)
        nmi = needle(mi, 2, 95)
        if mi % 12 == 0 :
            hr = (hr + 1) % 60
            cvs.delete(nhr)
            nhr = needle(hr, 3, 75)
    cvs.delete(nsec)
    nsec = needle(sec, 1, 100)
    root.after(1, draw)
draw()
tk.mainloop()


### 19.6 gif 애니메이션

이미지 중에서 .gif 형식은 짧은 애니메이션을 지원한다. 이러한 애니메이션은 각각의 프레임 이미지를 따로 저장한 후에 이를 지원하는 웹브라우저등에서 연속적으로 이 화면을 교체함으로서 이뤄진다.  하지만 tkinter 에서 이러한 움직임을 보이려면 각각의 프레임을 직접 교체시켜줘야한다. 
우선 애니메이션의 각 프레임단위의 억세스를 원할 경우 PIL Image 의 seek메소드를 사용할 수 있다. 

```python 
gif = Image.open('animate.gif')
for i in range(gif.n_frames):
    gif.seek(i)
    y,x =  int(i/col)*300 + 200, (i%col) * 300 + 100
    img += [PhotoImage(gif)]
    cvs.create_image(x, y, image = img[i])    
```

다음 코드는 간단한 애니메이션의 각 프레임을 격자형태로 출력한다. 


In [81]:
from PIL.ImageTk import PhotoImage


root = tk.Tk() 
cvs = tk.Canvas(root, width=1200, height=1200, bg = 'white')
cvs.pack() 
gif = Image.open('animate.gif')

img, col = [], 4

for i in range(gif.n_frames):
    gif.seek(i)
    y,x =  int(i/col)*300 + 200, (i%col) * 300 + 100
    img += [PhotoImage(gif)]
    cvs.create_image(x, y, image = img[i])

tk.mainloop() 


이번에는 버튼을 누를때마다 프레임이 바뀌도록 해본다. 이 경우 각 프레임을 배열로 저장해둬야 한다. 

```python 
gif = Image.open("animate2.gif")
imglist = []
for i in range(gif.n_frames):
    gif.seek(i)
    imglist += [PhotoImage(gif)]
```

버튼의 이미지의 변경은 위젯의 configure 메소드를 사용한다.  다음 코드는 i번째 프레임으로 버튼의 이미지를 변경한다. 

```Python 
btn.configure(image = imagelist[i])
```

이제 이것을 버튼 클릭 이벤트와 합치면 다음과 같다. 

In [82]:

from PIL import Image
from PIL.ImageTk import PhotoImage

root = tk.Tk()
gif = Image.open("animate2.gif")

imglist = []
for i in range(gif.n_frames):
    gif.seek(i)
    imglist += [PhotoImage(gif)]
    
btn = tk.Button(root, width = 400, height = 400, image = imglist[0])
btn.pack() 

frame = 0
def click(e):
    global imglist,frame
    frame = (frame + 1) % gif.n_frames    
    btn.configure( image = imglist[frame])
    
btn.bind('<1>', click)
tk.mainloop()


Canvas 에서 create_image로 생성된 이미지의 변경은 cvs.itemconfig 메소드로 가능하다. 
```python 
cvimg = cvs.create_image(200,200,imamge= imagelist[0])
cvs.itemconfig(cvimg, image = imagelist[frame])   
```

이를 응용해 타이머기능을 이용해 canvas 상의 gif를 움직이는 코드는 다음과 같다. 


In [147]:
from PIL import ImageTk, Image
import tkinter as tk 

root = tk.Tk()
cvs = tk.Canvas(root, width = 400, height = 400)
cvs.pack() 

gif = Image.open('animate2.gif')
imagelist = [] 
for i in range(gif.n_frames):
    gif.seek(i)    
    imagelist += [ImageTk.PhotoImage(gif)]

      
frame = 0 
cvimg = cvs.create_image(200,200, image = imagelist[0])   

def animate():
    global gif,imagelist,frame
    frame = (frame + 1) % gif.n_frames
    cvs.itemconfig(cvimg, image = imagelist[frame])   
    root.after(50, animate)

animate()
tk.mainloop()